# Introduction: Landmarks

In [1]:
import deltascope as ds
import deltascope.alignment as ut

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from scipy.optimize import minimize

import os
import tqdm
import json
import time

# Import raw data
The user needs to specify the directories containing the data of interest. Each sample type should have a key which corresponds to the directory path. Additionally, each object should have a list that includes the channels of interest.

In [2]:
# --------------------------------
# -------- User input ------------
# --------------------------------

data = {
    # Specify sample type key
    'wt': {
        # Specify path to data directory
        'path': './data/Output_wt03-09-21-29/',
        # Specify which channels are in the directory and are of interest
        'channels': ['AT','ZRF']
    },
    'you-too': {
        'path': './data/Output_yot03-09-23-21/',
        'channels': ['AT','ZRF']
    }
}

We'll generate a list of pairs of stypes and channels for ease of use.

In [3]:
data_pairs = []
for s in data.keys():
    for c in data[s]['channels']:
        data_pairs.append((s,c))

We can now read in all datafiles specified by the `data` dictionary above.

In [4]:
D = {}
for s in data.keys():
    D[s] = {}
    for c in data[s]['channels']:
        D[s][c] = ds.read_psi_to_dict(data[s]['path'],c)

100%|██████████| 69/69 [00:23<00:00,  1.78it/s]


# Calculate landmark bins
Based on the analysis above, we can select the optimal value of alpha bins.

In [5]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Pick an integer value for bin number based on results above
anum = 25

# Specify the percentiles which will be used to calculate landmarks
percbins = [50]

Calculate landmark bins based on user input parameters and the previously specified control sample.

In [6]:
theta_step = np.pi/4

In [7]:
lm = ds.landmarks(percbins=percbins, rnull=np.nan)
lm.calc_bins(D['wt']['AT'], anum, theta_step)

print('Alpha bins')
print(lm.acbins)
print('Theta bins')
print(lm.tbins)

Alpha bins
[-81.64435845 -74.84066191 -68.03696537 -61.23326883 -54.4295723
 -47.62587576 -40.82217922 -34.01848269 -27.21478615 -20.41108961
 -13.60739307  -6.80369654   0.           6.80369654  13.60739307
  20.41108961  27.21478615  34.01848269  40.82217922  47.62587576
  54.4295723   61.23326883  68.03696537  74.84066191  81.64435845]
Theta bins
[-3.14159265 -2.35619449 -1.57079633 -0.78539816  0.          0.78539816
  1.57079633  2.35619449  3.14159265]


# Calculate landmarks

In [8]:
lmdf = pd.DataFrame()

# Loop through each pair of stype and channels
for s,c in tqdm.tqdm(data_pairs):
    print(s,c)
    # Calculate landmarks for each sample with this data pair
    for k,df in tqdm.tqdm(D[s][c].items()):
        lmdf = lm.calc_perc(df, k, '-'.join([s,c]), lmdf)
        
# Set timestamp for saving data
tstamp = time.strftime("%m-%d-%H-%M",time.localtime())
        
# Save completed landmarks to a csv file
lmdf.to_csv(tstamp+'_landmarks.csv')
print('Landmarks saved to csv')

# Save landmark bins to json file
bins = {
    'acbins':list(lm.acbins),
    'tbins':list(lm.tbins)
}
with open(tstamp+'_landmarks_bins.json', 'w') as outfile:
    json.dump(bins, outfile)
print('Bins saved to json')

  0%|          | 0/4 [00:00<?, ?it/s]

wt AT



 25%|██▌       | 1/4 [00:21<01:03, 21.29s/it]

wt ZRF



 50%|█████     | 2/4 [00:41<00:41, 20.99s/it]

you-too AT



 75%|███████▌  | 3/4 [01:00<00:20, 20.47s/it]

you-too ZRF



100%|██████████| 4/4 [01:20<00:00, 20.31s/it]


Landmarks saved to csv
Bins saved to json
